# Tarea 07 - Chatbot RAG

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [14]:
import os
import re
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer

MAX_ARTICLES = 10

## Parte 1: Preparación del Corpus y Cálculo de Embeddings

### 1. Elección del Dataset

Se escogió el dataset de `BBC Full Text Document Classification` del sitio web de [Kaggle](https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification).

Se usarán los datos referentes a tecnología.

In [28]:
# Extraemos el contenido de los artículos
def extract_articles(folder_path: str) -> list[str]:
	articles = []
	for filename in os.listdir(folder_path)[:MAX_ARTICLES]:
		with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
			content = file.read()
			articles.append("content")
	return articles

### 2. Preprocesamiento

In [40]:
# Limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-zA-Záéíóúñü\s.]', '', text)  # Eliminar caracteres especiales, manteniendo puntos
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios extra
    return text

# Segmentar el texto en párrafos de 4 oraciones
def segment_text(text, sentences_per_fragment=4):
    # Dividir el texto en oraciones
    sentences = text.split(".")
    start = 0
    fragments = []
    # Separa el texto en la cantidad de oraciones
    while start < len(sentences):
        end = start + sentences_per_fragment
        fragments.append(". ".join(sentences[start:end]))
        start = end
    return fragments

### 3. Generación de Embeddings

In [46]:
def generate_embeddings(articles):
	pc = Pinecone(api_key="81002466-39d5-4599-a684-9b08a1c70aee")
	plain_articles = [paragraph for article in articles for paragraph in article]

	embeddings = pc.inference.embed(
		model="multilingual-e5-large",
		inputs=plain_articles,
		parameters={"input_type": "passage", "truncate": "END"}
	)

---

In [47]:
articles = extract_articles("./dataset/bbc-fulltext (document classification)/bbc/tech")
articles = list(map(clean_text, articles))
articles = list(map(segment_text, articles[:1]))
generate_embeddings(articles)